In [1]:
import pandas as pd
import numpy as np

In [123]:
# df = pd.read_csv("./mapping/raw_data/unimelb_dookie.csv") #Dookie
# df = pd.read_csv("./mapping/raw_data/unimelb_burnley.csv") #Burnley
# df = pd.read_csv("./mapping/raw_data/unimelb_creswick.csv") #Creswick
# df = pd.read_csv("./mapping/raw_data/unimelb_sheppartion.csv") #Sheppartion
# df = pd.read_csv("./mapping/raw_data/unimelb_southbank.csv") #Southbank
df = pd.read_csv("./mapping/raw_data/unimelb_werribee.csv") #Werribee
# df = pd.read_csv("./mapping/raw_data/unimelb_parkville.csv") #Parville

In [124]:
df[['Building', 'z', 'Floor label prefix','Floorplan label']]= df['Building;z;Floor label prefix;Floorplan label'].str.split(";",expand=True)

In [125]:
df.drop(['Building;z;Floor label prefix;Floorplan label','z','Floor label prefix','Floorplan label'],axis =1,inplace=True)

In [126]:
df.drop_duplicates(subset=['Building'],inplace=True)

In [127]:
#Parkville only
# df.drop(index=df.index[0],inplace=True)

In [128]:
df[['Building name', 'No', 'id']]= df['Building'].str.split("(",expand=True)

In [129]:
df.drop(['Building','id'],axis =1,inplace=True)

In [130]:
df.dropna(axis=0,inplace=True)

In [131]:
df['No'] =df['No'].str.replace(')', '').str.replace('BUILDING', '').astype(str)


C:\Users\amorntiyangg\AppData\Local\Temp\ipykernel_16836\2192794832.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['No'] =df['No'].str.replace(')', '').str.replace('BUILDING', '').astype(str)


In [132]:
#Parkville
# df['No'] =df['No'].str.replace('Building', '').astype(str)
# df['No'] =df['No'].str.replace('BUILIDNG', '').astype(str)

In [133]:
#Create campus id mapping
campus_id_mapping = pd.DataFrame(data={'Campus_code': ['WER', 'DOO','SPT','STH','BUR','CRE','PAR'], 'campus_id': [217, 220,221,216,218,219,200]})

In [134]:
# Look up the campus_id for a given campus_code
# campus_code = 'DOO'
campus_code = 'WER'
# campus_code = 'STH'
# campus_code = 'SPT'
# campus_code = 'BUR'
# campus_code = 'CRE'
# campus_code = 'PAR'
campus_id = campus_id_mapping.loc[campus_id_mapping['Campus_code'] == campus_code, 'campus_id'].iloc[0]

In [135]:
# Define a lambda function to update the URL string
def update_url(row):
    return 'https://use.mazemap.com/?campusid={}&sharepoitype=identifier&sharepoi={}'.format(campus_id, row['No'])

# Apply the lambda function to the "no" column and assign the results to a new column "url"
df['url'] = df.apply(update_url, axis=1)

Mapping building name

In [136]:
mapping_naming = pd.read_excel("Buidling_mapping_Mazemap.xlsx")

In [137]:
# rename the 'Building Code' column to 'No'
mapping_naming = mapping_naming.rename(columns={'Building Code': 'No'})

In [138]:
df['No'] =df['No'].str.strip()
mapping_naming['No'] =mapping_naming['No'].str.strip()

In [139]:
# create a dictionary from the mapping DataFrame
mapping_dict = dict(zip(mapping_naming['No'], mapping_naming['Building Name']))


In [140]:
# use the map function to replace the building names in the original DataFrame
df['Building name'] = df['No'].map(mapping_dict)
df

,Building name,No,url
0,WERRIBEE VETERINARY HOSPITAL,411,https://use.mazemap.com/?campusid=217&sharepoi...
414,WERRIBEE PATHOLOGY BUILDING,416,https://use.mazemap.com/?campusid=217&sharepoi...
510,WERRIBEE PARASITOLOGY BUILDING,417,https://use.mazemap.com/?campusid=217&sharepoi...
551,WERRIBEE LEARNING & TEACHING BUILDING,418,https://use.mazemap.com/?campusid=217&sharepoi...
626,WERRIBEE CAMPUS SERVICES WORKSHOP/STORE,421,https://use.mazemap.com/?campusid=217&sharepoi...
628,WERRIBEE RESEARCH LABORATORY,434,https://use.mazemap.com/?campusid=217&sharepoi...
633,AVIAN RESEARCH,437,https://use.mazemap.com/?campusid=217&sharepoi...


In [141]:
#check the mapping key
df_building_codes = set(df['No'])
mapping_dict_codes = set(mapping_dict.keys())

In [142]:
print(df_building_codes - mapping_dict_codes)  # check the building codes in df that are not in the mapping_dict
print(mapping_dict_codes - df_building_codes)  # check the building codes in mapping_dict that are not in df

set()
{'522', '533', '636', '479', '871', '700', '814', '745', '656', '678', '428', '290', '271', '6855', '865', 'EXT-W502', '410I', '556', 'EXT-W400', '864', '168', '498', '733', '148B', '199', '130', '897', '982', '410C', '120', '110', '184', '609', '932', '220', '222', '862', '993', '262', '213', '548', '244', '582', '395', '576', 'EXT-W503', '581', 'EXT-B905', '169', '186', '442', '469', '642', '867', '990', '410', '940', '571', '410J', '812', '818', '992', '422', '472', '6858', '810', '675', '519', '983', '6852', '585', '630', '384', '156', '701', '106', '509', '695', '930', '410U', '998', '140', '260', '378', '6966', '901', '583', '103', '478', '115', '295', '554', '815', '588', '938', '256', '615', '721', '605', '335', '474', '216', '710', '725', 'EXT-W600', '263', '651', '243', '950', '5031', '703', 'EXT-P160', '410N', '665', '628', '514', '203', '5020', '734', '214', '683', '5593', '564', '5002', '873', '570', '906', '291', '696', '5051', '607', '207', '722', '536', '544', '73

In [143]:
# df.to_excel("./mapping/unimelb_dookie_url.xlsx", index=False) 
# df.to_excel("./mapping/unimelb_burnley_url.xlsx", index=False) 
# df.to_excel("./mapping/unimelb_creswick_url.xlsx", index=False) 
# df.to_excel("./mapping/unimelb_sheppartion_url.xlsx", index=False) 
# df.to_excel("./mapping/unimelb_southbank_url.xlsx", index=False) 
# df.to_excel("./mapping/unimelb_werribee_url.xlsx", index=False) 
# df.to_excel("./mapping/unimelb_parkville_url.xlsx", index=False) 